In [36]:
import pandas as pd
import numpy as np
from scipy import fftpack
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from scipy.fftpack import fft
from sklearn.model_selection import cross_validate,train_test_split,StratifiedKFold
from sklearn import metrics
import pickle
from scipy.stats import skew

def DataPreProcessing(CGM_Data):
    no_of_rows=CGM_Data.shape[0]
    no_of_columns = CGM_Data.shape[1]
    CGM_Data.dropna(axis=0, how='all', thresh=no_of_columns/4, subset=None, inplace=True)
    CGM_Data.dropna(axis=1, how='all', thresh=no_of_rows/4, subset=None, inplace=True)
    CGM_Data.interpolate(axis=1, method ='linear', limit_direction ='forward', inplace=True)
    CGM_Data.bfill(axis=1,inplace=True)
    return CGM_Data


def ExtractFeatures(CGM_Data):
    Feature_Matrix = pd.DataFrame()

    # Feature 1 - Windowed Mean (for 30 min interval)
    win_size=6
    total_vals = CGM_Data.shape[1]-win_size
    for index in range(0, total_vals, win_size):
        dm = CGM_Data.iloc[:, index:index + win_size].mean(axis=1)
        Feature_Matrix['Mean ' + str(index)] = dm

    print(Feature_Matrix.shape)
    
    # Feature 2 - Windowed Standard Deviation (for 30 min interval)
    win_size=6
    total_vals = CGM_Data.shape[1]-win_size
    for index in range(0, total_vals, win_size):
        dstd = CGM_Data.iloc[:, index:index + win_size].std(axis=1)
        Feature_Matrix['Std_deviation ' + str(index)] = dstd
        
    print(Feature_Matrix.shape)
    
    # Feature 3 - Fast Fourier Transform
    FFT = pd.DataFrame()
    def calculate_fft_vals(series):
        FFT_abs = abs(fft(series))
        FFT_abs.sort()
        return np.flip(FFT_abs)[0:8]

    FFT['FFT_vals'] = CGM_Data.apply(lambda series: calculate_fft_vals(series), axis=1)
    FFT_Vals= pd.DataFrame(FFT.FFT_vals.tolist(), columns=['FFT1', 'FFT2', 'FFT3', 'FFT4', 'FFT5', 'FFT6', 'FFT7','FFT8'],index=FFT.FFT_vals.index)
    Feature_Matrix = pd.concat([Feature_Matrix,FFT_Vals],axis=1)
    print(Feature_Matrix.shape)
    
    
    # Feature 4 - CGM Velocity 
    Velocity_Data = pd.DataFrame()
    win_size=6
    total_vals=CGM_Data.shape[1]-win_size

    for index in range(0, total_vals):
        dv = CGM_Data.iloc[:, index + win_size] - CGM_Data.iloc[:, index]
        Velocity_Data['vel'+str(index)] = dv

    Feature_Matrix['Max CGM Vel']=Velocity_Data.max(axis = 1,skipna=True)
       
    print(Feature_Matrix.shape)
    
    
    # Feature 5 - Skewness
    def calculate_skewness(series):
        series_counts = series.value_counts()
        skewness_vals = skew(series_counts)
        return skewness_vals

    Feature_Matrix['skewness'] = CGM_Data.apply(lambda row: calculate_skewness(row), axis=1)
    
    print(Feature_Matrix.shape)
    
    # Feature 6 - polyfit   
    def calculate_polyfit(series,degree=3):
        row_arr = np.array(series.index)
        return np.polyfit(row_arr, series, degree)
    
    Polyfit_vals = CGM_Data.apply(calculate_polyfit,axis=1,result_type='expand')
    Feature_Matrix = pd.concat([Feature_Matrix,Polyfit_vals],axis=1)
    
    print(Feature_Matrix.shape)
    
    
    return Feature_Matrix


def predit_class():
    
    K=5
    print("------ Predicting class using KNN model--------")
    for i in range(1,K+1):
        handler = open("KNN"+str(i)+".model","rb")
        model = pickle.load(handler)
        pd.DataFrame(model.predict(Feature_Matrix)).to_csv("KNN"+str(i)+"output.csv",index=False,header=False)
        handler.close()
        print("Generated output for KNN"+str(i)+" in KNN"+str(i)+"output.csv")
    

    print("------ Predicting class using SVM model--------")
    handler = open("SVC.model","rb")
    model = pickle.load(handler)
    pd.DataFrame(model.predict(Feature_Matrix)).to_csv("SVCoutput.csv",index=False,header=False)
    handler.close()
    print("Generated output for SVC in SVCoutput.csv")
        
    
    
    
if __name__=="__main__":
    column_names = [i for i in range(0,31)]
    input_file_path = input("Enter the test file name/path")
    print("Entered file path",input_file_path)
    Test_Data = pd.read_csv(input_file_path,names=column_names)
    
    # Data Preprocessing
    Test_Data_Pre = DataPreProcessing(Test_Data)
    
    #Extract Features
    Test_Data_Features = ExtractFeatures(Test_Data_Pre)
    
    # Standardize feature matrix
    Feature_Matrix  = StandardScaler().fit_transform(Test_Data_Features)
    
    # Predict class labels
    predit_class()
    

Enter the test file name/pathNomeal1.csv
Entered file path Nomeal1.csv
(51, 4)
(51, 8)
(51, 16)
(51, 17)
(51, 18)
(51, 22)
------ Predicting class using KNN model--------
Generated output for KNN1 in KNN1output.csv
Generated output for KNN2 in KNN2output.csv
Generated output for KNN3 in KNN3output.csv
Generated output for KNN4 in KNN4output.csv
Generated output for KNN5 in KNN5output.csv
------ Predicting class using SVM model--------
Generated output for SVC in SVCoutput.csv
